In [ ]:
#choropleth v1 
#importing saved csv as to not run nomainatim again
cs_df = pd.read_csv("HeatCSV.csv")
cs_df.drop(cs_df.loc[cs_df['Country']!='United States'].index, inplace=True)
cs_df = cs_df.dropna()
cs_df["County"] = cs_df['County'].str.replace(' County','')
#count the number of searchers in each state and each US county
state_list = Counter(cs_df['State'])
county_list = Counter(cs_df['County'])
#sorting the states and counties alphabetically and making them into dataframes
sorted_states = pd.DataFrame(sorted(state_list.items()), columns = ["State", "Searchers"])
sorted_counties = pd.DataFrame(sorted(county_list.items()), columns = ["County", "Searchers"])
#creating a choropleth map for all us counties based on the amount of searchers there
#importing the spatial geojson that maps the US counties
us_counties = r'counties.geojson'
with open(us_counties) as usa_file:
    us_counties = geojson.load(usa_file)
#choropleth for all us counties, filling in the county based on the number of searchers located there
folium.Choropleth(
    geo_data = us_counties,
    data = sorted_counties,
    name = "Choropleth",
    columns = ['County', 'Searchers'],
    key_on= 'feature.properties.NAME',
    fill_color = 'YlOrRd',
    fill_opacity = 1,
    line_opacity = 0.2,
    legend_name = "Searchers",
    smooth_factor = 0,
    Highlight = True,
    line_color = "#0000",
    show=False,
    overlay=True,
    control = True,
    nan_fill_color = "White").add_to(county_map)

In [ ]:
# adding hover functionality for a choropleth
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = final_df,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','Total Ranking', 'Ease of Business ', 'Size', 'Growth'],
        aliases=['Name','Total Ranking', 'Ease of Business Ranking', 'Size Ranking', 'Growth Ranking'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
map_3.add_child(NIL)
map_3.keep_in_front(NIL)

In [ ]:
#saving the mapping file
#outfp = "county_mapv3.html"
#county_map.save(outfp)

In [ ]:
#MSA CHOROPLETH V1
#Reading in MSA DATA
geo_data = gpd.read_file('tl_2019_us_cbsa.json')
cities = pd.read_excel('CitiesRankings.xlsx')
cities['MSA'] = cities['MSA'].str.replace(" MSA", "")
cities = cities.rename(columns={"MSA": "NAME"})
cities_set = set(cities["NAME"])
data_set = set(geo_data['NAME'])
#finding mismatch MSA's
c = list(cities_set.difference(data_set))
#MSA Choropleth
#finding the index in the cities df of all mismatched MSA's
city_index = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city = cities.loc[cities["NAME"].str.contains(i, case = False)]
    city_index.append(city["NAME"].index[0])
#finding the proper names of the MSA's given by the US Census Data
city_names = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city_name = geo_data.loc[geo_data["NAME"].str.contains(i, case = False)]
    #print(city_name["NAME"].index[0])
    city_name = city_name.iloc[0].NAME
    city_names.append(city_name)
#replacing all values in the cities df with the correct name
count = 0
for i in city_index:
    cities.at[i, "NAME"] = city_names[count]
    count += 1
#choropleth for all US MSA's 
choropleth = folium.Choropleth(
    geo_data = geo_data,
    data = final_df,
    name = "Cities Choropleth",
    columns = ['CBSAFP', 'Total Ranking'],
    key_on= 'feature.properties.CBSAFP',
    fill_color = 'YlOrRd',
    fill_opacity = 1,
    line_opacity = 0.2,
    legend_name = "Choropleth Legend: MSA Ranking",
    smooth_factor = 0,
    Highlight = True,
    line_color = "#0000",
    show=False,
    overlay=True,
    control = True,
    nan_fill_color = "White").add_to(map_3)

In [ ]:
#geolocating for the searchers
#using nominatim to geocode the location of each searcher based on the latitude and longitude
geolocator = Nominatim(user_agent = "geoapiexercise")
searching_states = []
searching_country = []
searching_county = []
for i in range(df.shape[0]):
    latitude = str(df.iloc[i]['search lat'])
    longitude = str(df.iloc[i]['search long'])
    
    location = geolocator.reverse(latitude + "," + longitude)
    
    address = location.raw['address']
    
    state = address.get('state')
    country = address.get('country')
    county = address.get('county')
    
    searching_states.append(state)  
    searching_country.append(country)
    searching_county.append(county)
#counting the amount of searchers in each us state and each us county
state_list = Counter(searching_states)
county_list = Counter(searching_county)
#zipping together the county, searching state, and country for each searchfund listed in the database
state_county = list(zip(searching_county, searching_states, searching_country))
#creating a dataframe that contains the county, state and country for each searcher. Then cleaning the data
cs_df = pd.DataFrame(state_county, columns = ["County", "State", "Country"])
cs_df.to_csv("HeatCSV.csv")


In [ ]:
#version 1 of the popup
for i in range(df.shape[1]):
    
        location = [df.iloc[i]["search lat"],df.iloc[i]['search long']]
        tooltip = "Searchfund: {}<br> Current Status: {}<br> Click for more".format(df.iloc[i]["name"], df.iloc[i]['stage'])
    
        folium.Marker(location, # adding more details to the popup screen using HTML
                      popup="""
                      <i>Date Formed: </i> <br> <b>{}</b> <br> 
                      <i>Company Acquired (If Applicable): </i><b><br>{}</b><br>
                      <i>Search Length (In Days): </i><b><br>{}</b><br>""".format(
                        df.iloc[i]['formed'], 
                        df.iloc[i]['company'], 
                        df.iloc[i]['Search Length']), 
                      tooltip=tooltip).add_to(marker_cluster)
        #adds a radius around the searcher point
        #folium.Circle(location, radius = 4000).add_to(county_map)
#version 2 of the popup
for i in range(df.shape[1]):
    try:
        location = [df.iloc[i]['search lat'],df.iloc[i]['search long']]
        tooltip = "Searchfund: {}<br> Current Status: {}<br> Click for more".format(df["name"][i], df['stage'][i])
    
        folium.Marker(location, # adding more details to the popup screen using HTML
                      popup = Popup("""
                      <i>Date Formed: </i> <br> <b>{}</b> <br> 
                      <i>Company Acquired (If Applicable): </i><b><br>{}</b><br>
                      <i>Search Length (In Days): </i><b><br>{}</b><br>""".format(
                        df.iloc[i]['formed'], 
                        df.iloc[i]['company'], 
                        df.iloc[i]['Search Length'])),
                     tooltip = tooltip).add_to(marker_cluster)
        #adds a radius around the searcher point
        #folium.Circle(location, radius = 4000).add_to(county_map)
    except ValueError:
        print("NAN value")
        pass
    except:
        print("other error")
        pass

In [ ]:

m2_search = MarkerCluster(name = 'Cluster of Searchers', control = True).add_to(map_2)
m2_acq = MarkerCluster(name = 'Cluster of Acquisitions', control = True).add_to(map_2)
cluster_1 = MarkerCluster(name = "Cluster of Starbucks", control = False).add_to(map_2)
cluster_2 = MarkerCluster(name = "Cluster of Apple Stores", control = False).add_to(map_2)
cluster_3 = MarkerCluster(name = "Cluster of Whole Foods", control = False).add_to(map_2)
#adding markers for all starbucks, apple stores, and whole foods in the US
for point in range(0, len(starbucks_loc)):
    tooltip = "Company"
    try:
        folium.Marker(starbucks_loc[point], popup = "<br> <b>Starbucks</b> <br>" , tooltip = tooltip, icon=folium.Icon(icon = "coffee", color = 'pink', icon_color='white', prefix = 'fa'), overlay = True).add_to(cluster_1)
    except KeyError:
        print("KeyError")
for point in range(0, len(apple_loc)):
    try:
        folium.Marker(apple_loc[point], popup = "<br> <b>Apple Store</b> <br>" , icon=folium.Icon(icon = "apple", color = 'black', icon_color='white', prefix = 'fa')).add_to(cluster_2)
    except KeyError:
        print("KeyError")
for point in range(0, len(whole_loc)):
    try:
        folium.Marker(whole_loc[point], popup = "<br> <b>Whole Foods</b> <br>" , icon=folium.Icon(icon = "shopping-bag", color = 'lightgreen', icon_color='white', prefix = 'fa')).add_to(cluster_3)
    except KeyError:
        print("KeyError")

In [ ]:
#acquisition clusters v2
for i in range(0, len(acquisition_locations)):
    html=f"""
        <h1 style="font-size:20px"> Company Acquired: {acq_df.iloc[i]['company']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Industry: <b><br>{acq_df.iloc[i]['industry']}</b><br> </li>
            <li>Acquired By: <b><br>{acq_df.iloc[i]['name']}</b><br> </li>
            <li>Date of Acquisition: <b><br>{df.iloc[i]['acquired']}</b><br></li>
        </ul>
        """
    tooltip = """
        <p> style="font-size: 11px"> Company Acquired: {acq_df.iloc[i]['company']}</p>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(acquisition_locations[i], popup = popup , tooltip = tooltip, icon=folium.Icon(color='red', prefix = 'fa', icon_color = 'white', icon = 'check')).add_to(m2_acq)

In [ ]:

m2_search = MarkerCluster(name = 'Cluster of Searchers', control = True).add_to(map_2)
m2_acq = MarkerCluster(name = 'Cluster of Acquisitions', control = True).add_to(map_2)
cluster_1 = MarkerCluster(name = "Cluster of Starbucks", control = False).add_to(map_2)
cluster_2 = MarkerCluster(name = "Cluster of Apple Stores", control = False).add_to(map_2)
cluster_3 = MarkerCluster(name = "Cluster of Whole Foods", control = False).add_to(map_2)
#adding toggle layers to the county mapS
#folium.LayerControl().add_to(county_map)
#simple example of adding the searchfund locations with a blue icon popup#Search locations v3
#adding html to searchers
# add marker one by one on the map
for i in range(0, len(search_locations)):
    html=f"""
        <h1 style="font-size:20px"> Searchfund : {df.iloc[i]['name']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Founded: <b><br>{df.iloc[i]['formed']}</b><br> </li>
            <li>Stage: <b><br>{df.iloc[i]['stage']}</b><br></li>
            <li>Company Acquired(If Applicable): <b><br>{df.iloc[i]['company']}</b><br></li>
            <li>Date of Acquisition: <b><br>{df.iloc[i]['acquired']}</b><br></li>
            <li>Lenght of Search (In Days): <b><br>{df.iloc[i]['Search Length']}</b><br></li>
        </ul>
        </p>
        <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=search_locations[i],
        popup=popup,
        icon=folium.DivIcon(html=f"""
            <div><svg>
                <circle cx="50" cy="50" r="40" fill="#69b3a2" opacity=".4"/>
                <rect x="35", y="35" width="30" height="30", fill="red", opacity=".3" 
            </svg></div>""")
    ).add_to(m2_search)

In [ ]:
#us counties choropleth data cleaning
fips = us.states.mapping('fips', 'name')
count_series = cs_df.groupby(['County', 'STATEFP', 'Country']).size()
count_series = count_series.to_frame(name = 'Searchers').reset_index()
gdf = gpd.read_file('counties.geojson')
gdf = gdf.rename(columns = {'NAME':'County'})
new_df = pd.merge(gdf, count_series,  how='left', left_on=['STATEFP','County'], right_on = ['STATEFP','County'])
new_df['State'] = new_df['STATEFP'].map(fips)
new_df.head()
fips_2 = us.states.mapping('name', 'fips')
fips_2 = us.states.mapping('name', 'fips')
cs_df['STATEFP'] = cs_df['State'].map(fips_2)

In [ ]:
#adding cluster for all MSA's that rank in top 25
msa_cluster = MarkerCluster(name = "Top 25 MSAs").add_to(map_2)

#adding a point for all the top 25 cities in the MSAs
for point in range(0, len(centers)):
    if 0 <final_df.iloc[point]["Total Ranking"] <= 25.0:
        try:
            folium.Marker(centers[point], popup = "<br> <b>MSA </b> <br>" , icon=folium.Icon(icon = "star", color = 'orange', icon_color='white', prefix = 'fa'), overlay = True).add_to(msa_cluster)
        except KeyError:
            pass


In [ ]:
#MSA CHOROPLETH
#Reading in MSA DATA
geo_data = gpd.read_file('tl_2019_us_cbsa.json')
cities = pd.read_excel('CitiesRankings.xlsx')
cities['MSA'] = cities['MSA'].str.replace(" MSA", "")
cities = cities.rename(columns={"MSA": "NAME"})
cities_set = set(cities["NAME"])
data_set = set(geo_data['NAME'])
#finding mismatch MSA's
c = list(cities_set.difference(data_set))

In [ ]:
#MSA Choropleth
#finding the index in the cities df of all mismatched MSA's
city_index = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city = cities.loc[cities["NAME"].str.contains(i, case = False)]
    city_index.append(city["NAME"].index[0])
#finding the proper names of the MSA's given by the US Census Data
city_names = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city_name = geo_data.loc[geo_data["NAME"].str.contains(i, case = False)]
    #print(city_name["NAME"].index[0])
    city_name = city_name.iloc[0].NAME
    city_names.append(city_name)
#replacing all values in the cities df with the correct name
count = 0
for i in city_index:
    cities.at[i, "NAME"] = city_names[count]
    count += 1
final_df = pd.merge(geo_data, cities, on = "NAME", how = 'outer')
final_df = final_df.fillna(0)
MSA_cluster = MarkerCluster(name = "MSA Cluster").add_to(map_3)

In [ ]:
#choropleth for all US MSA's 
choropleth = folium.Choropleth(
    geo_data = geo_data,
    data = final_df,
    name = "Cities Choropleth",
    columns = ['CBSAFP', 'Total Ranking'],
    key_on= 'feature.properties.CBSAFP',
    fill_color = 'Greys',
    fill_opacity = 1,
    line_opacity = 0.2,
    legend_name = "Choropleth Legend: MSA Ranking",
    smooth_factor = 0,
    Highlight = True,
    line_color = "#0000",
    show=False,
    overlay=True,
    control = True,
    nan_fill_color = "White").add_to(map_3)

In [ ]:
# adding hover functionality for the MSA's
style_function = lambda x: {'fillColor': 'red' if x['properties']['Total Ranking'] > 25.0 
                            else 'green' if 0 < x['properties']['Total Ranking'] <= 25 else 'white','fillOpacity':0.75, 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = json_final,
    style_function=style_function, 
    control=False,
    name = "MSA Cities Hover Tool",
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','Total Ranking', 'Ease of Business ', 'Size', 'Growth'],
        aliases=['Name','Total Ranking', 'Ease of Business Ranking', 'Size Ranking', 'Growth Ranking'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
map_3.add_child(NIL)
map_3.keep_in_front(NIL)

In [ ]:
final_df = final_df.sort_values(by=['Total Ranking'])
final_df = final_df.fillna(0)
ranking_list = list(final_df['Total Ranking'])
ranking_list = ranking_list[:357]

In [ ]:
#MSA LINEAR Geojson map
colormap = cm.LinearColormap(["green", "yellow", "orange", "red"], vmin = 1, vmax = final_df["Total Ranking"].max())
colormap.caption = "Ranking of US MSAs"
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Total Ranking']) if 0 < x['properties']['Total Ranking'] <= 357.0 else 'white'
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL=folium.features.GeoJson(
        data = json_final,
        style_function=style_function,
        name = "MSA Hover Tool",
        control=True,
        overlay = True,
        show = False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NAME','Total Ranking', 'Ease of Business ', 'Size', 'Growth'],
            aliases=['Name','Total Ranking', 'Ease of Business Ranking', 'Size Ranking', 'Growth Ranking'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
colormap.add_to(map_2)
map_2.add_child(NIL)

In [1]:
#creating a heatmap based on the longitude and latitude of the searchers in searchfunder
#this is removed to create space
heat_map = folium.Map(location = [40.692943, -73.985880], default_zoom_start = 7)
search_lat = df['search lat'].tolist()
search_long = df['search long'].tolist()
heat_zip = zip(search_lat, search_long)
HeatMap(heat_zip, name = "HeatMap", min_opacity = .5, max_zoom = 22, radius = 15, blur = 8, gradient = {0.3: 'blue', 0.65: "lime", 1: 'red'}, overlay = True, control = True, show = False).add_to(county_map)

NameError: name 'folium' is not defined

In [ ]:
#choropleth v2
cs_df = pd.read_csv("HeatCSV.csv")
cs_df.drop(cs_df.loc[cs_df['Country']!='United States'].index, inplace=True)
cs_df = cs_df.dropna()
cs_df["County"] = cs_df['County'].str.replace(' County','')
fips = us.states.mapping('fips', 'name')
fips_2 = us.states.mapping('name', 'fips')
cs_df['STATEFP'] = cs_df['State'].map(fips_2)
#counting the occurences of a county in the dataframe, one count stands for one searcher in the county
count_series = cs_df.groupby(['County', 'STATEFP', 'Country']).size()
count_series = count_series.to_frame(name = 'Searchers').reset_index()
gdf = gpd.read_file('counties.geojson')
gdf = gdf.rename(columns = {'NAME':'County'})
new_df = pd.merge(gdf, count_series,  how='left', left_on=['STATEFP','County'], right_on = ['STATEFP','County'])
new_df['State'] = new_df['STATEFP'].map(fips)
new_df["Searchers"] = new_df["Searchers"].fillna(0)
new_df["geometry"] = new_df["geometry"].simplify(tolerance = 0.9)
#new_df = new_df.simplify(tolerance = 0.5)

county_json = new_df.to_json()